In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ======================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [2]:
import os
import glob

import cudf
import numpy as np
import pandas as pd

import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")


### Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = os.environ.get("NUM_ROWS", 100000)

In [5]:
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., int(NUM_ROWS)).astype(np.int32), 1, 50000)
# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, int(NUM_ROWS)), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['age_days'] = np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)
df['weekday_sin']= np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [6]:
df.head()

,session_id,item_id,category,age_days,weekday_sin,day
0,84190,16,4,0.223325,0.121436,4
1,74994,18,4,0.055975,0.285401,4
2,80080,29,6,0.301125,0.274779,3
3,87793,4,1,0.110288,0.653482,7
4,78898,18,4,0.586547,0.859649,6


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers. Note that the `Categorify` op encodes OOVs or nulls to `0` automatically. In our synthetic dataset we do not have any nulls. On the other hand `0` is also used for padding the sequences in input block, therefore, you can set `start_index=1` arg in the Categorify op if you want the encoded null or OOV values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

Here our goal is to create sequential features. To do so, we are grouping the features together at the session level in the following cell. In this synthetically generated example dataset, we do not have a timestamp column, but if we had one (that's the case for most real-world datasets), we would be sorting the interactions by the timestamp column as in this [example notebook](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/end-to-end-session-based/01-ETL-with-NVTabular.ipynb). Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [7]:
SESSIONS_MAX_LENGTH =20

# Categorify categorical features
categ_feats = ['item_id', 'category'] >> nvt.ops.Categorify()

# Define Groupby Workflow
groupby_feats = categ_feats + ['session_id', 'day', 'age_days', 'weekday_sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "age_days": ["list"],
        'weekday_sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['category-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH, pad=True) 
)

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH, pad=True) 
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['age_days-list', 'weekday_sin-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH, pad=True) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['item_id-list', 'category-list', 'age_days-list', 'weekday_sin-list'] >>  nvt.ops.ValueCount()

workflow = nvt.Workflow(filtered_sessions['session_id', 'day-first'] + seq_feats_list)

dataset = nvt.Dataset(df)

# Generate statistics for the features and export parquet files
# this step will generate the schema.pbtxt file
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [9]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,day-first,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-list,"(Tags.ID, Tags.CATEGORICAL, Tags.ITEM, Tags.LI...","DType(name='int64', element_type=<ElementType....",True,False,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,502.0,item_id,503.0,52.0,20.0,20.0
3,category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,False,NaN,0.0,0.0,0.0,.//categories/unique.category.parquet,0.0,184.0,category,185.0,30.0,20.0,20.0
4,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,20.0
5,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,20.0


Save NVTabular workflow.

In [10]:
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow_etl"))

## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [11]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))

In [14]:
# Read in the processed parquet file
sessions_gdf = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))

In [17]:
print(sessions_gdf.head(3))

   session_id  day-first                                       item_id-list  \
0       70000          5  [3, 10, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...   
1       70001          2  [21, 18, 61, 70, 41, 15, 47, 2, 54, 33, 16, 6,...   
2       70002          4  [92, 190, 78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   

                                       category-list  \
0  [1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [5, 5, 17, 18, 10, 6, 13, 1, 16, 4, 6, 2, 0, 0...   
2  [25, 55, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                       age_days-list  \
0  [0.6733318, 0.36617512, 0.56633216, 0.0, 0.0, ...   
1  [0.2618201, 0.47376513, 0.4943239, 0.47170892,...   
2  [0.7606575, 0.68139946, 0.7614335, 0.0, 0.0, 0...   

                                    weekday_sin-list  
0  [0.007507112, 0.7772118, 0.89882964, 0.0, 0.0,...  
1  [0.9721299, 0.9868918, 0.58013266, 0.54288113,...  
2  [0.13897721, 0.026063459, 0.5617346, 0.0, 0.0,...  


In [18]:
from transformers4rec.utils.data_utils import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 17.49it/s]


## Checking the preprocessed outputs

In [19]:
TRAIN_PATHS = os.path.join(OUTPUT_DIR, "1", "train.parquet")

In [20]:
df = pd.read_parquet(TRAIN_PATHS)
df.head()

,session_id,item_id-list,category-list,age_days-list,weekday_sin-list
0,70004,"[55, 5, 25, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[16, 1, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0.092122145, 0.95085436, 0.43779024, 0.880414...","[0.84441566, 0.566204, 0.74057275, 0.42418396,..."
1,70007,"[30, 7, 10, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9, 4, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.87233526, 0.6098228, 0.80743825, 0.89595354...","[0.7175466, 0.12882064, 0.7239913, 0.53173864,..."
2,70024,"[4, 91, 12, 1, 18, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[2, 24, 4, 1, 5, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,...","[0.028581467, 0.56572497, 0.43579134, 0.434222...","[0.84453183, 0.7441511, 0.34946096, 0.35230267..."
4,70051,"[1, 14, 201, 14, 93, 42, 0, 0, 0, 0, 0, 0, 0, ...","[1, 3, 54, 3, 28, 10, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.5466232, 0.24969716, 0.17476603, 0.39066973...","[0.9378631, 0.5088918, 0.36846048, 0.71432567,..."
5,70059,"[22, 10, 8, 14, 24, 15, 12, 133, 0, 0, 0, 0, 0...","[5, 3, 2, 3, 7, 6, 4, 35, 0, 0, 0, 0, 0, 0, 0,...","[0.6290009, 0.77528137, 0.68838376, 0.5259807,...","[0.10275719, 0.42096895, 0.2630285, 0.31103444..."


In [21]:
import gc
del df
gc.collect()

594

You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model. Please shut down this kernel to free the GPU memory before you start the next one.